In [1]:
import pandas as pd 
import numpy as np 
from bs4 import BeautifulSoup
from selenium import webdriver
import requests

In [2]:
pip install uszipcode


Note: you may need to restart the kernel to use updated packages.


In [3]:
from uszipcode import SearchEngine

search = SearchEngine(simple_zipcode=True)

In [4]:
#main function that will take the epicenter for disaster, calculate area of affect, find all zipcodes in that area
#and calculate a projected damage cost based on median home value and number of homes in area. with a penalty reduction
#assuming not all homes were destroyed.
def main_search(Zip, disaster_type):
    try:
    #setting dictionary for area covered by disaster, in KM   
        disaster_dist = {
            'hurricane': 100,
            'flood': 50,
            'fire':.5
        }
    #setting the effect ratio based on research from last 5 disasters by type, 
    #(More info needed for more accurate calculations)
        disaster_effect = {
            'hurricane': .1,
            'flood' : .81,
            'fire': 1
        }
        #instantiate the search engine
        search = SearchEngine(simple_zipcode=True)
        #searching for epicenter zipcode
        zipcode = search.by_zipcode(str(Zip))
        #Setting result as a list with every zipcode within a certain range. 
        result = search.by_coordinates(zipcode.lat,zipcode.lng, radius= (disaster_dist[disaster_type]), returns=1000)
        #printing the median household value for epicenter zipcode 
        print(f'The median house value of {Zip} ({zipcode.major_city}) is: {zipcode.median_home_value}')
        #iterate through our results and getting median home value times number of housing units in each zipcode effected by hurricane 
        value = 0
        for i in result:

            try: 
            #Try and except statements to account for null values where no houses exist.
                value += i.median_home_value * i.housing_units
            except:
                pass
        #Printing the total damage done minus the penalty for the specific disaster. 
        total_value = value * disaster_effect[disaster_type]
        function_total(total_value), print(f'for a {disaster_type}')
        
    except:
        print('Zipcode was not found/ was invalid')

In [5]:
main_search('21201', 'flood')

The median house value of 21201 (Baltimore) is: 208500
Zipcode was not found/ was invalid


In [6]:
def function_total(total_damage):
    convert = int(total_damage)
    formater = format(convert, ',')
    
    if len(formater.split(',')) == 3:
        print(f'Estimated damage between {int(formater[0]) * .75} and {formater[0]} Million dollars')
    elif len(formater.split(',')) == 2:
        print(f'Estimated damage between {int(formater[0]) * .75} and {formater[0]} Thousand dollars') 
    elif len(formater.split(',')) == 4:
        print(f'Estimated damage between {int(formater[0]) * .75} and {formater[0]} Billion dollars')  
    elif total_damage >= 1_000_000_000_000:
        print(f'Estimated damage over 1 trillion dollars')
    else: print(f'Estimated damage: {total_damage * .75}-{total_damage} dollars')    

In [7]:
function_total(1000000000)

Estimated damage between 0.75 and 1 Billion dollars


In [10]:
all_zip = {}

In [11]:
new_zip = search.by_zipcode(10001)

In [12]:
new_zip.area_code_list

['718', '917', '347', '646']

In [13]:
for i in range( 10000,99999):
    new_zip = search.by_zipcode(str(i))
    if new_zip.population != None:
        all_zip[i] = {'latt'              : new_zip.lat,
                      'long'              : new_zip.lng,
                      'median_house_value': new_zip.median_home_value,
                      'housing_units'     : new_zip.housing_units,
                      'population'        : new_zip.population}

In [14]:
all_zip = pd.DataFrame(all_zip).T

In [15]:
all_zip.shape

(30562, 5)

In [16]:
all_zip.head()

,housing_units,latt,long,median_house_value,population
10001,12476.0,40.750,-73.990,650200.0,21102.0
10002,34541.0,40.720,-73.990,535600.0,81410.0
10003,31078.0,40.730,-73.990,817700.0,56024.0
10004,2197.0,40.700,-74.020,894200.0,3089.0
10005,5317.0,40.705,-74.005,1000001.0,7135.0


In [17]:
all_zip.to_csv('./all_zipcodes.csv')

In [73]:
#main function that will take the epicenter for disaster, calculate area of affect, find all zipcodes in that area
#and calculate a projected damage cost based on median home value and number of homes in area. with a penalty reduction
#assuming not all homes were destroyed.
def main_search2(Zip, disaster_type, damage_radius=None, damage_percent=None):
    try:
    #setting dictionary for area covered by disaster, in KM   
        disaster_dist = {
            'hurricane': 100,
            'flood': 50,
            'fire':.5
        }
    #setting the effect ratio based on research from last 5 disasters by type, 
    #(More info needed for more accurate calculations)
        disaster_effect = {
            'hurricane': .1,
            'flood' : .81,
            'fire': 1
        }
        if damage_radius == None:
            radius = disaster_dist[disaster_type]
        else:
            radius = damage_radius
        if damage_percent == None:
            damage = disaster_effect[disaster_type]
        else:
            damage = damage_percent
            
        #instantiate the search engine
        search = SearchEngine(simple_zipcode=True)
        #searching for epicenter zipcode
        zipcode = search.by_zipcode(str(Zip))
        #Setting result as a list with every zipcode within a certain range. 
        result = search.by_coordinates(zipcode.lat,zipcode.lng, radius= radius, returns=25)
        #printing the median household value for epicenter zipcode 
        print(f'The median house value of {Zip} ({zipcode.major_city}) is: {zipcode.median_home_value}')
        #iterate through our results and getting median home value times number of housing units in each zipcode effected by hurricane 
        value = 0
        
        for i in result:
            zipcode = i.zipcode
            try: 
            #Try and except statements to account for null values where no houses exist.
                
                value += all_zip['median_house_value'][int(zipcode)] * all_zip['housing_units'][int(zipcode)]
                
            except:
                pass
        #Printing the total damage done minus the penalty for the specific disaster. 
        total_value = value * damage
        function_total(total_value)
        
    except KeyError:
        print('Zipcode was not found/ was invalid')

In [69]:
all_zip.fillna(0,inplace=True)

In [75]:
main_search2('21201', 'hurricane', damage_percent=0.5, damage_radius=10)

The median house value of 21201 (Baltimore) is: 208500
Estimated damage between 2.25 and 3 Billion dollars


In [59]:
list2

NameError: name 'list2' is not defined

In [25]:
all_zip['median_house_value'][21202]

202200.0

In [ ]:
Session.rollback()

In [27]:
zipcode = search.by_zipcode(21201)

In [38]:

result = search.by_coordinates(zipcode.lat,zipcode.lng, radius= 30, returns=1000)
value = 0
for i in result:
    print(all_zip['median_house_value'][int(i.zipcode)])

208500.0
202200.0
269800.0
80200.0
112900.0
224600.0
55900.0
192900.0
152900.0
102700.0
108600.0
136100.0
441700.0
176500.0
222600.0
185000.0
131200.0
183100.0
162400.0
279100.0
195000.0
158500.0
291100.0
280500.0
155800.0
227600.0
296400.0
424300.0
226400.0
268500.0
213500.0
240200.0
306100.0
nan
216400.0
247900.0
325200.0
195300.0
242100.0
460400.0
368200.0
231300.0
351100.0
251200.0
236000.0
202100.0
342000.0
345000.0
341000.0
447900.0
308000.0
339200.0
279600.0
288100.0
nan
387300.0
372700.0
467500.0
394700.0
562600.0
475200.0
434100.0
221900.0
nan
310200.0
561800.0
306700.0
382800.0
391400.0
340700.0
477400.0
286900.0
370100.0
428700.0
288600.0
310600.0
388500.0
449400.0
259200.0
839300.0
389800.0
nan
660200.0
704100.0
671000.0
386700.0
388800.0
285800.0
557500.0
359400.0
308100.0
176200.0
317300.0
656500.0
567300.0
703200.0
362500.0
396500.0
758000.0
323200.0
406300.0
255000.0
739600.0
369800.0
299300.0
429100.0
698700.0
nan
300200.0
650200.0
368800.0
634400.0
272900.0
475900.0
4